In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('dati supernovae.csv')

In [8]:
df

,# nome_SN,redshift,mod_dist,mod_dist_err,unknown
0,1993ah,0.028488,"353,465,833,928",0.223905932998,0.128419
1,1993ag,0.050043,"366,823,679,154",0.166828851413,0.128419
2,1993o,0.052926,"368,176,912,545",0.1557559148,0.128419
3,1993b,0.070086,"374,467,365,424",0.158466934433,0.128419
4,1992bs,0.062668,"374,834,093,505",0.156099434739,0.128419
...,...,...,...,...,...
580,# alpha 0.121851859725,NaN,NaN,NaN,NaN
581,# beta 2.46569277393,NaN,NaN,NaN,NaN
582,# delta -0.0363405630486,NaN,NaN,NaN,NaN
583,# M(h=0.7 statistical only) -19.3182761161,NaN,NaN,NaN,NaN
